# For assigning chemical info and partial charges to PDB structures (saves to well-defined SDF files)

## Imports

In [1]:
# Generic Imports
from collections import defaultdict

# File I/O
from pathlib import Path
from tqdm.notebook import tqdm

# Cheminformatics
from openff.toolkit.topology import Topology, Molecule

# Static Paths
MONO_DIR = Path('monomer_files_dup')
PDB_DIR  = Path('pdb_files_dup')

## Chemistries

In [2]:
# defining reacting functional groups
reaction_pairs = {
    'NIPU' : ('cyclocarbonate', 'amine'),
    'urethane' : ('isocyanate', 'hydroxyl')
}
# chemistries = ('urethane', 'NIPU')
chemistries = [i for i in reaction_pairs.keys()]

## Obtaining complete PDB-monomer pairs for iteration

In [3]:
chem_paths = defaultdict(defaultdict)
for chemistry in chemistries:
    chem_pdb  = PDB_DIR / chemistry
    chem_mono = MONO_DIR / chemistry

    has_pdb  = set(path.stem for path in chem_pdb.glob( '*.pdb'))
    has_mono = set(path.stem for path in chem_mono.glob('*.json'))

    for valid_mol_name in (has_pdb & has_mono):
        pdb_path  = chem_pdb  / f'{valid_mol_name}.pdb'
        mono_path = chem_mono / f'{valid_mol_name}.json'

        chem_paths[chemistry][valid_mol_name] = (pdb_path, mono_path)

## Paramaterizing and saving to SDF, recording mols where match fails

In [4]:
from polymerist.openfftools import topology
from polymerist.openfftools.pcharge import MolCharger

from polymerist.monomers import MonomerGroup
from polymerist.residues.partition import partition


topo_dir = Path('Topologies_dup')
topo_dir.mkdir(exist_ok=True)

charge_method = 'Espaloma-AM1-BCC'
charger = MolCharger.subclass_registry[charge_method]()


unmatched_pdb_mols = defaultdict(defaultdict)
for chemistry, path_dict in chem_paths.items():
    chem_dir = topo_dir / chemistry
    chem_dir.mkdir(exist_ok=True)

    progress = tqdm(path_dict.items())
    for mol_name, (pdb_path, mono_path) in progress:
        progress.set_description_str(f'{chemistry} : {mol_name}')
        try:
            progress.set_postfix_str('Loading files')
            monogrp = MonomerGroup.from_file(mono_path)
            offtop = Topology.from_pdb(pdb_path, _custom_substructures=monogrp.monomers)
            progress.set_postfix_str('Partitioning')
            was_partitioned = partition(offtop)
            assert(was_partitioned)

            progress.set_postfix_str('Assigning Charges')
            offmol = topology.get_largest_offmol(offtop)
            offmol.name = mol_name
            cmol = charger.charge_molecule(offmol)

            progress.set_postfix_str('Saving SDF')
            offtop = cmol.to_topology()
            sdf_path = chem_dir / f'{mol_name}.sdf'
            topology.topology_to_sdf(sdf_path, offtop)
        except Exception as e:
            print(f'{mol_name} : {e}')

        # except UnmatchedAtomsError:
        #     offtop_unmatched, *_ = Topology.from_pdb_and_monomer_info(
        #         str(pdb_path),
        #         monomer_info_json=mono_path,
        #         strict=False,
        #         verbose=False
        #     )
        #     offmol_unmatched = next(offtop_unmatched.molecules)
        #     unmatched_pdb_mols[chemistry][mol_name] = offmol_unmatched


/home/timber/miniconda3/envs/polymerist_env/lib/python3.11/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/timber/miniconda3/envs/polymerist_env/lib/python3.11/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/timber/miniconda3/envs/polymerist_env/lib/python3.11/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/timber/miniconda3/envs/polymerist_env/lib/python3.11/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


  0%|          | 0/6 [00:00<?, ?it/s]

/home/timber/miniconda3/envs/polymerist_env/lib/python3.11/site-packages/espaloma_charge/openff_wrapper.py:112: IncorrectNumConformersWarning: Molecule 'Molecule with name 'NIPU_10' and SMILES '[H]c1c(c(c(c(c1C(c2c(c(c(c(c2[H])[H])OC([H])([H])[C@]([H])(C([H])([H])OC(=O)C(=O)OC([H])([H])[C@@]([H])(C([H])([H])Oc3c(c(c(c(c3[H])[H])C(c4c(c(c(c(c4[H])[H])OC([H])([H])[C@@]([H])(C([H])([H])OC(=O)N([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])N([H])C(=O)OC([H])([H])[C@@]([H])(C([H])([H])Oc5c(c(c(c(c5[H])[H])C(c6c(c(c(c(c6[H])[H])OC([H])([H])[C@@]([H])(C([H])([H])OC(=O)N([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])N([H])C(=O)OC([H])([H])[C@@]([H])(C([H])([H])Oc7c(c(c(c(c7[H])[H])C(c8c(c(c(c(c8[H])[H])OC([H])([H])[C@@]([H])(C([H])([H])OC(=O)N([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H

  0%|          | 0/60 [00:00<?, ?it/s]

/home/timber/miniconda3/envs/polymerist_env/lib/python3.11/site-packages/espaloma_charge/openff_wrapper.py:112: IncorrectNumConformersWarning: Molecule 'Molecule with name 'urethane_9' and SMILES '[H]c1c(c(c(c(c1[H])[H])N(C([H])([H])C([H])([H])O[H])C([H])([H])C([H])([H])OOC([H])([H])C([H])([H])N(c2c(c(c(c(c2[H])[H])[H])[H])[H])C([H])([H])C([H])([H])OC(=O)N([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])N([H])C(=O)OC([H])([H])C([H])([H])N(c3c(c(c(c(c3[H])[H])[H])[H])[H])C([H])([H])C([H])([H])OC(=O)N([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])N([H])C(=O)OC([H])([H])C([H])([H])N(c4c(c(c(c(c4[H])[H])[H])[H])[H])C([H])([H])C([H])([H])OC(=O)N([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])N([H])C(=O)OC([H])([H])C([H])([H])N(c5c(c(c(c(c5[H])[H])[H])[H])[H])C([H])([H])C([H])([H])OC(=O)N([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])N([H])C(=O)OC([H])([H])C([H])([H])N(c6c(c(c(c(c6[H])[H])[H])[H])[H])C([H])([H])C([H])([H])OC(=O)N([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])N([H])C(=

...

/home/timber/miniconda3/envs/polymerist_env/lib/python3.11/site-packages/espaloma_charge/openff_wrapper.py:112: IncorrectNumConformersWarning: Molecule 'Molecule with name 'urethane_52' and SMILES '[H]C([H])(C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])OOC([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])OC(=O)N([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])N([H])C(=O)OC([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])OC(=O)N([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])N([H])C(=O)OC([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])OC(=O)N([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])N([H])C(=O)OC([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([

urethane_22 : Programming error: OpenEye atom stereochemistry assumptions failed. The atom in the oemol has stereochemistry None and the atom in the offmol has stereochemistry S.


/home/timber/miniconda3/envs/polymerist_env/lib/python3.11/site-packages/espaloma_charge/openff_wrapper.py:112: IncorrectNumConformersWarning: Molecule 'Molecule with name 'urethane_38' with bad SMILES and Hill formula 'C495H387N36O99P9'' has 1 conformers, but charge method 'espaloma-am1bcc' expects exactly 0.
  self._check_n_conformers(


urethane_38 : Programming error: OpenEye atom stereochemistry assumptions failed. The atom in the oemol has stereochemistry None and the atom in the offmol has stereochemistry S.


/home/timber/miniconda3/envs/polymerist_env/lib/python3.11/site-packages/espaloma_charge/openff_wrapper.py:112: IncorrectNumConformersWarning: Molecule 'Molecule with name 'urethane_55' and SMILES '[H]C1(C(C(C(C(C1([H])O[H])([H])[H])([H])[H])([H])OOC2(C(C(C(C(C2([H])[H])([H])[H])([H])OC(=O)N([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])N([H])C(=O)OC3(C(C(C(C(C3([H])[H])([H])[H])([H])OC(=O)N([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])N([H])C(=O)OC4(C(C(C(C(C4([H])[H])([H])[H])([H])OC(=O)N([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])N([H])C(=O)OC5(C(C(C(C(C5([H])[H])([H])[H])([H])OC(=O)N([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])N([H])C(=O)OC6(C(C(C(C(C6([H])[H])([H])[H])([H])OC(=O)N([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])N([H])C(=O)OC7(C(C(C(C(C7([H])[H])([H])[H])([H])OC(=O)N([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])N([H])C

## Probing PDB mismatches

In [ ]:
unmatched_pdb_mols['urethane']['urethane_13']

In [ ]:
for mol_name, mol in unmatched_pdb_mols['urethane'].items():
    mol.name = mol_name

mols_by_len = sorted(unmatched_pdb_mols['urethane'].values(), key=lambda mol : mol.n_atoms)

In [ ]:
mol = mols_by_len[-1]

rdmol = mol.to_rdkit()
for i, atom in enumerate(mol.atoms):
    if not atom.metadata['already_matched']:
        print(atom.metadata)
        rdmol.GetAtomWithIdx(i).SetProp('atomNote', 'unmatched')

display(rdmol)